# Login Authentication Mock Data

Notebook de desenvolvimento responável por realizar processo de mock dos dados e evenviar os dados para o eventhub utilizando autenticação com SPN


<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://raw.githubusercontent.com/Foiac/MobileFraudDetectSolution/main/Editaveis/Imagens/login-authetication-mock-data.png" alt="SparkStreaming Ingest" style="width: 800px">
</div>

#### Import dependecies

In [10]:
import random
import json
from faker import Faker
from datetime import datetime
from azure.eventhub import EventHubProducerClient, EventData
from azure.identity import ClientSecretCredential
import pandas as pd
import threading

#### Path definitions and secrets

In [ ]:
azure_tenant_id: str = "dc25df03-ffa5-4111-b188-46fe6cd26a3a"
azure_client_id: str = "7060731f-a210-43a3-8217-e6741e224f03"
azure_client_secret: str = "" 
eventhub_namespace: str = "fraud-eh-namespace"
eventhub_namespace_fully: str = f"{eventhub_namespace}.servicebus.windows.net"
eventhub_name: str = "fraud-detect"

In [12]:
file_path = 'phonedataset.csv'

df = pd.read_csv(file_path, sep=';')
phone_list = df.values.tolist()

with open('city_clusters.json', 'r', encoding='utf-8') as file:
    city_clusters = json.load(file)

#### Initial Definitions

In [13]:
start_date = datetime(2024, 11, 1, 0)
end_date = datetime(2024, 11, 2, 23)

In [14]:
fake = Faker()

transaction_prob = [0.78, 0.22] 
app_version_prob = [0.05, 0.20, 0.75]  
error_prob = [0.90, 0.08, 0.02] 
uids = [str(fake.random_number(digits=11, fix_len=True)) for _ in range(1000)]

#### Function to Create SPN Client 

In [15]:
def spn_authentication():
    
    print("init procces")
    credential = ClientSecretCredential(tenant_id=azure_tenant_id, 
                                        client_id=azure_client_id,
                                        client_secret=azure_client_secret)
    print("credential create")
    producer = EventHubProducerClient(fully_qualified_namespace=eventhub_namespace_fully,
                                        eventhub_name=eventhub_name,
                                        credential=credential)
    print("producer create")

    return producer

#### Function to Generate Random Data

In [16]:
def generate_random_data():
    """Gera um dicionário com dados aleatórios no formato JSON especificado."""

    city = random.choice(city_clusters)
    latitude = random.gauss(city["lat"], city["std_dev"])
    longitude = random.gauss(city["long"], city["std_dev"])

    brand_version = random.choice(phone_list)
    phone_brand = brand_version[0]
    operational_version = brand_version[1]

    uid = random.choice(uids) if random.random() < 0.01 else str(fake.random_number(digits=11, fix_len=True))

    return {
        "imei": str(fake.random_number(digits=15, fix_len=True)),
        "mac": ':'.join([f"{random.randint(0, 255):02X}" for _ in range(6)]),
        "network": random.choice(["VIVO", "TIM", "CLARO", "OI"]),
        "client_ip": fake.ipv4(),
        "latitude": latitude,
        "longitude": longitude,
        "uid": uid,
        "password": fake.password(length=6, special_chars=False, digits=True, upper_case=True, lower_case=True),
        "transaction": random.choices(["true", "false"], weights=transaction_prob, k=1)[0],
        "api": "login-authentication",
        "endpoint": "v1/login",
        "os": operational_version,
        "phone_brand": phone_brand,
        "app_version": random.choices(["2.0.1", "2.0.2", "2.0.3"], weights=app_version_prob, k=1)[0],
        "error": random.choices(["0", "INCORRECT_PASS", "USER_NOT_FOUND"], weights=error_prob, k=1)[0],
        "timestamp": str(fake.unix_time(start_datetime=start_date, end_datetime=end_date) * 1000)
    }

#### Function to Send Data do Eventhub

In [17]:
def send_message_to_eventhub(eh_producer, msg_number, trh_number):
    """Abre client com eventhub e envia batch de mensagens."""
    for i in range(msg_number):

        message = json.dumps(generate_random_data())

        try:
            with eh_producer:
                event_data_batch = eh_producer.create_batch()

                event_data_batch.add(EventData(message))

                eh_producer.send_batch(event_data_batch)
                
                print(f"[{trh_number}] - Mensagem enviada com sucesso! - {message}")
        except ValueError as ve:
            print(f"Erro ao enviar mensagem: {ve}")
        except Exception as e:
            print(f"Erro inesperado ao enviar mensagem: {e}")
        finally:
            eh_producer.close()


#### Start Thread to Send Data

In [19]:
eh_producer = spn_authentication()

msg_number = 200

thread_1 = threading.Thread(target=send_message_to_eventhub, args = (eh_producer, msg_number, 1))
thread_2 = threading.Thread(target=send_message_to_eventhub, args = (eh_producer, msg_number, 2))

thread_1.start()
thread_2.start()

thread_1.join()
thread_2.join()

init procces
credential create
producer create
[1] - Mensagem enviada com sucesso! - {"imei": "871771247913125", "mac": "B2:C3:77:9E:48:7F", "network": "TIM", "client_ip": "18.87.201.3", "latitude": -22.597137673814498, "longitude": -47.16256350595047, "uid": "42044546347", "password": "F46Cwj", "transaction": "true", "api": "login-authentication", "endpoint": "v1/login", "os": "Android 13 (MIUI 14)", "phone_brand": "Poco X5 Pro", "app_version": "2.0.3", "error": "0", "timestamp": "1730552149000.0"}
[2] - Mensagem enviada com sucesso! - {"imei": "531353552928884", "mac": "3D:94:DB:7E:CB:33", "network": "CLARO", "client_ip": "213.105.104.246", "latitude": -9.44253039339551, "longitude": -63.30026875465339, "uid": "14020677548", "password": "4CypLg", "transaction": "true", "api": "login-authentication", "endpoint": "v1/login", "os": "Android 14 (Stock Android)", "phone_brand": "Google Pixel 8 Pro", "app_version": "2.0.2", "error": "0", "timestamp": "1730477698000.0"}
[1] - Mensagem envia